# Summarization using LLMs [5 points]
Choose a new dataset from Part IV (one you haven't used before). Repeat the steps from Part IV, and deploy the resulting model.

## Steps:

1. Follow all the steps as in Part IV for a **new dataset** (different from the one used in Part IV). Provide a brief analysis of the results. You are welcome to reuse your code from Part IV.

   **Dataset Selection (New Dataset from Part IV List):**

<span style='color:green'>### YOUR ANSWER ###</span>

We are using billSum Dataset

   **Step 1: Dataset preparation and preprocessing (as in Part IV):**

In [ ]:
### ADD YOUR CODE HERE ###
# Reusing code from Part IV - Dataset Preparation and Preprocessing

!pip install datasets
!pip install transformers
!pip install rouge_score
!pip install sacrebleu
!pip install bert_score
!pip install evaluate

from datasets import load_dataset



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00


In [ ]:
bill_sum = load_dataset("billsum",trust_remote_code=True)
print(bill_sum)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/91.8M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

ca_test-00000-of-00001.parquet:   0%|          | 0.00/6.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 18949
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 3269
    })
    ca_test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1237
    })
})


In [ ]:
from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")


def tokenize_and_separate(example):

    doc_tokens = tokenizer(
        example["text"],
        padding="longest",
        truncation=True,
        max_length=512
    )
    summary_tokens = tokenizer(
        example["summary"],
        padding="longest",
        truncation=True,
        max_length=256
    )
    return {
        "doc_input_ids": doc_tokens["input_ids"],
        "doc_attention_mask": doc_tokens["attention_mask"],
        "summary_input_ids": summary_tokens["input_ids"],
        "summary_attention_mask": summary_tokens["attention_mask"]
    }


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

In [ ]:
doc_tokens_train = bill_sum["train"].select(range(2000)).map(tokenize_and_separate, batched=True)


doc_tokens_val = bill_sum["train"].select(range(2001, 2201)).map(tokenize_and_separate, batched=True)


doc_tokens_test = bill_sum["test"].select(range(100)).map(tokenize_and_separate, batched=True)

print(doc_tokens_train[1:10])
print(doc_tokens_val[1:10])
print(doc_tokens_test[1:10])

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'text': ["SECTION 1. SHORT TITLE.\n\n    This Act may be cited as the ``Human Rights Information Act''.\n\nSEC. 2. FINDINGS.\n\n    Congress finds the following:\n            (1) The people of the United States consider the national \n        and international protection and promotion of human rights and \n        the rule of law the most important values of any democracy. The \n        founding fathers defined human rights prominently in the Bill \n        of Rights, giving those rights a special priority and \n        protection in the Constitution.\n            (2) Federal agencies are in possession of documents \n        pertaining to gross human rights violations abroad which are \n        needed by foreign authorities to document, investigate, and \n        subsequently prosecute instances of continued and systematic \n        gross human rights violations, including those directed against \n        citizens of the United States.\n            (3) The United States will continue 

In [ ]:
doc_tokens_train.save_to_disk("tokenized_bill_sum_train")
doc_tokens_val.save_to_disk("tokenized_bill_sum_val")
doc_tokens_test.save_to_disk("tokenized_bill_sum_test")

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/200 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk


train_dataset = load_from_disk("tokenized_bill_sum_train")
val_dataset = load_from_disk("tokenized_bill_sum_val")
test_dataset = load_from_disk("tokenized_bill_sum_test")


   **Step 2: Model Fine-Tuning (as in Part IV):**

In [ ]:
from transformers import (
    BartTokenizer,
    BartForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    TrainerCallback
)
import torch
import numpy as np
import evaluate
import os


class PrintMetricsCallback(TrainerCallback):
    def __init__(self):
        self.epoch_metrics = []

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        print("\n Evaluation Metrics at Epoch", int(state.epoch))

        print(f"  • ROUGE-1         : {metrics.get('eval_rouge1', 0):.2f}")
        print(f"  • ROUGE-2         : {metrics.get('eval_rouge2', 0):.2f}")
        print(f"  • ROUGE-L         : {metrics.get('eval_rougeL', 0):.2f}")
        print(f"  • ROUGE-Lsum      : {metrics.get('eval_rougeLsum', 0):.2f}")
        print(f"  • BLEU            : {metrics.get('eval_bleu', 0):.2f}")
        print(f"  • BERTScore F1    : {metrics.get('eval_bertscore_f1', 0):.2f}")

        self.epoch_metrics.append({
            "epoch": state.epoch,

            "rouge1": metrics.get("eval_rouge1", 0),
            "rouge2": metrics.get("eval_rouge2", 0),
            "rougeL": metrics.get("eval_rougeL", 0),
            "bleu": metrics.get("eval_bleu", 0),
            "bertscore_f1": metrics.get("eval_bertscore_f1", 0)
        })

# Main training function
def train_bart_model(train_dataset, val_dataset, learning_rate, batch_size, num_epochs, gradient_accumulation_steps):

    print(f"Training for learning rate: {learning_rate}, batch size: {batch_size}, epochs: {num_epochs}")
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("Using CUDA device")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
        print("Using MPS device (Apple Silicon)")
    else:
        device = torch.device("cpu")
        print("Using CPU device")

    tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
    model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
    model.to(device)

    rouge = evaluate.load("rouge")
    bleu = evaluate.load("sacrebleu")
    bertscore = evaluate.load("bertscore")

    def preprocess_dataset(dataset, tokenizer):
        return dataset.map(lambda x: {
            "input_ids": x["doc_input_ids"],
            "attention_mask": x["doc_attention_mask"],
            "labels": [
                (token if token != tokenizer.pad_token_id else -100)
                for token in x["summary_input_ids"]
            ]
        }, remove_columns=dataset.column_names)

    train_dataset = preprocess_dataset(train_dataset, tokenizer)
    val_dataset = preprocess_dataset(val_dataset, tokenizer)

    def compute_metrics(eval_preds):
        preds, labels = eval_preds
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
        bleu_result = bleu.compute(predictions=decoded_preds, references=[[ref] for ref in decoded_labels])
        bert_result = bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en")

        result = {
            **rouge_result,
            "bleu": bleu_result["score"],
            "bertscore_f1": np.mean(bert_result["f1"]) * 100
        }
        return {f"eval_{k}": round(v, 4) for k, v in result.items()}

    class CustomSeq2SeqTrainer(Seq2SeqTrainer):
        def __init__(self, *args, tokenizer=None, **kwargs):
            super().__init__(*args, **kwargs)
            self._custom_tokenizer = tokenizer

        def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
            if prediction_loss_only:
                return super().prediction_step(model, inputs, prediction_loss_only, ignore_keys)
            inputs = self._prepare_inputs(inputs)
            generated_tokens = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_length=128,
                num_beams=4
            )
            labels = inputs.get("labels")
            if labels is not None:
                max_len = labels.shape[1]
                generated_tokens = self._pad_tensors_to_max_len(generated_tokens, max_len)
            return (None, generated_tokens, labels)

        def _pad_tensors_to_max_len(self, tensor, max_len):
            pad_token_id = self._custom_tokenizer.pad_token_id
            padded = pad_token_id * torch.ones(
                (tensor.shape[0], max_len),
                dtype=tensor.dtype,
                device=tensor.device
            )
            padded[:, :tensor.shape[1]] = tensor
            return padded

    training_args = Seq2SeqTrainingArguments(
        output_dir="./results",
        eval_strategy="epoch",
        logging_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        predict_with_generate=True,
        logging_dir="./logs",
        logging_steps=10,
        fp16=True,
        save_total_limit=1,
        gradient_accumulation_steps=gradient_accumulation_steps,
        save_strategy="epoch",
        report_to="none",
        remove_unused_columns=False
    )

    metrics_callback = PrintMetricsCallback()

    trainer = CustomSeq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[metrics_callback]
    )

    trainer.train()

    return trainer, model, metrics_callback


In [ ]:
trainer, model, metrics_callback = train_bart_model(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    learning_rate=5e-5,
    batch_size=4,
    num_epochs=10,
    gradient_accumulation_steps=4
)

Training for learning rate: 5e-05, batch size: 4, epochs: 10
Using CUDA device


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



 Evaluation Metrics at Epoch 1
  • ROUGE-1         : 0.44
  • ROUGE-2         : 0.24
  • ROUGE-L         : 0.32
  • ROUGE-Lsum      : 0.35
  • BLEU            : 12.68
  • BERTScore F1    : 87.30


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(



 Evaluation Metrics at Epoch 2
  • ROUGE-1         : 0.44
  • ROUGE-2         : 0.24
  • ROUGE-L         : 0.32
  • ROUGE-Lsum      : 0.35
  • BLEU            : 12.36
  • BERTScore F1    : 87.51

 Evaluation Metrics at Epoch 3
  • ROUGE-1         : 0.44
  • ROUGE-2         : 0.25
  • ROUGE-L         : 0.33
  • ROUGE-Lsum      : 0.36
  • BLEU            : 12.74
  • BERTScore F1    : 87.80

 Evaluation Metrics at Epoch 4
  • ROUGE-1         : 0.43
  • ROUGE-2         : 0.25
  • ROUGE-L         : 0.33
  • ROUGE-Lsum      : 0.36
  • BLEU            : 11.66
  • BERTScore F1    : 87.77

 Evaluation Metrics at Epoch 5
  • ROUGE-1         : 0.45
  • ROUGE-2         : 0.26
  • ROUGE-L         : 0.34
  • ROUGE-Lsum      : 0.37
  • BLEU            : 12.64
  • BERTScore F1    : 88.01

 Evaluation Metrics at Epoch 6
  • ROUGE-1         : 0.43
  • ROUGE-2         : 0.26
  • ROUGE-L         : 0.34
  • ROUGE-Lsum      : 0.36
  • BLEU            : 11.36
  • BERTScore F1    : 87.97

 Evaluation Metrics

   **Step 3: Evaluation and analysis (as in Part IV):**

In [ ]:
### ADD YOUR CODE HERE ###
# Reusing code from Part IV - Evaluation and Analysis

trainer.save_model("bart_model")


tokenizer.save_pretrained("bart_model")

('bart_model/tokenizer_config.json',
 'bart_model/special_tokens_map.json',
 'bart_model/vocab.json',
 'bart_model/merges.txt',
 'bart_model/added_tokens.json')

In [2]:
import zipfile
import os

def unzip_file(zip_path, extract_to_folder):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f"Extracted to: {extract_to_folder}")

# Example usage
unzip_file('bart_model_bonus.zip', 'bart_model')


Extracted to: bart_model


In [ ]:
from transformers import BartForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments

def preprocess_dataset(dataset, tokenizer):
        return dataset.map(lambda x: {
            "input_ids": x["doc_input_ids"],
            "attention_mask": x["doc_attention_mask"],
            "labels": [
                (token if token != tokenizer.pad_token_id else -100)
                for token in x["summary_input_ids"]
            ]
        }, remove_columns=dataset.column_names)


model = BartForConditionalGeneration.from_pretrained("bart_model")


training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    do_eval=True,
    predict_with_generate=True,
    per_device_eval_batch_size=4,
    logging_dir="./logs",
    report_to="none"
)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer
)



tokenizer = BartTokenizer.from_pretrained("bart_model")
print(test_dataset.column_names)
#test_dataset = preprocess_dataset(test_dataset, tokenizer)

print("Running predictions on the test set, gives score values")
test_predictions = trainer.predict(test_dataset)

decoded_preds = tokenizer.batch_decode(test_predictions.predictions, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(
    np.where(test_predictions.label_ids != -100, test_predictions.label_ids, tokenizer.pad_token_id),
    skip_special_tokens=True
)

import pandas as pd
df_results = pd.DataFrame({
    "Generated_Report": decoded_preds,
    "Reference_Report": decoded_labels
})
df_results.to_csv("test_results.csv", index=False)
print(" Saved test results to 'test_results.csv'")

print("\n Test Set Metrics:")
for key, val in test_predictions.metrics.items():
    print(f"{key}: {val:.4f}")

<ipython-input-11-d014960aa2b4>:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


['input_ids', 'attention_mask', 'labels']
Running predictions on the test set


 Saved test results to 'test_results.csv'

 Test Set Metrics:
test_loss: 2.0814
test_model_preparation_time: 0.0052
test_runtime: 16.2095
test_samples_per_second: 6.1690
test_steps_per_second: 1.5420


   **Brief Analysis of Results:**

The test loss of 2.0814 indicates moderate performance.The model struggles to generate accurate summaries on unseen dat .It can be due to insufficient training, complex legal text, or incorrect hyperparameters.Fine tuning with a better learning rate or longer training could help reduce this loss.

The test runtime of 16.2095 sec and 6.169 samples/sec show moderate processing speed.Due to time constraints we have only used a part of the dataset, hence we get a good training time

The  model works but shows substantial loss and moderate speed.
Legal text complexity in BillSum may require specialized fine-tuning.
Exploring larger BART variants or PEGASUS could yield better results.

2. Deploy your trained model. This could be a simple app that takes a text as input and returns a summary.

In [3]:
### ADD YOUR CODE HERE ###
from transformers import BartForConditionalGeneration,BartTokenizer
def generate_summary(input_text, model, tokenizer, max_input_length=1024, max_target_length=256):

    inputs = tokenizer(
        input_text,
        max_length=max_input_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    ).to(model.device)


    summary_ids = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_target_length,
        num_beams=4,
        early_stopping=True
    )
    #print(summary_ids)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

model = BartForConditionalGeneration.from_pretrained("bart_model")
tokenizer = BartTokenizer.from_pretrained("bart_model")
custom_doc = """As New Jersey’s population has grown, farmers and neighbors have been brought closer together.
Some aspects of this growth are positive, as farmers are able to cultivate new markets, and consumers
are able to access local agricultural products. Other aspects are more challenging, as residential,
municipal, and county concerns sometimes conflict with the needs of farm businesses. To help resolve
such issues, the Right to Farm Act, N.J.S.A. 4:1C-1 et seq., was signed into law in 1983.
Every state in the country has its own version of Right to Farm. In New Jersey, the Right to Farm Act is
designed to help address conflicts among farmers, neighbors, municipalities, and counties regarding a
farm’s practices. If a conflict cannot be resolved informally, such as through Agricultural Mediation or
other discussion by the parties, the Act provides for a formal review process. Under the Act, anyone
aggrieved by the operation of a commercial farm is required to file a complaint with the County
Agriculture Development Board (CADB) prior to filing an action in court. This is what makes New
Jersey’s Right to Farm Act so strong – commercial farms cannot be taken to court by neighbors and
local governments first. Rather, these complaints must first be heard by the CADB or State Agriculture
Development Committee (SADC), agencies that have special expertise in the agricultural industry and
understand the needs of farm operations."""
summary_output = generate_summary(custom_doc, model, tokenizer)
print("\n📘 Generated Summary:")
print(summary_output)



📘 Generated Summary:
Right to Farm Act - Amends the New Jersey Right of Agriculture Act to provide for the right of a farmer, neighbors, municipalities, and county to file a complaint with the County Agricultural Development Board (CADB) regarding the operation of a commercial farm. The Act requires that such complaints be heard by the CADB and the state Agriculture Development Committee (SADC). The Act also requires that a complaint be filed with the CADB prior to filing an action in court.


3. Record a short video (~5 mins) demonstrating how it works. Explain the key aspects of your implementation and deployment process in the video.
   a. Upload the video to UBbox and create a shared link
   b. Add the link at the end of your ipynb file.

   **Shared UBbox Video Link:**

<span style='color:green'>### YOUR ANSWER ###</span>

https://buffalo.box.com/s/9qat1t15qycpyxzyaa7m66selqbjiw4a

4. References. Include details on all the resources used to complete this part.

https://pytorch.org/torchtune/0.3/basics/tokenizers.html

https://huggingface.co/docs/transformers/en/model_doc/bart